In [1]:
import torch

# 标量的反向传播
x = torch.Tensor([2])

# autograd: set require_grad = True
w = torch.randn(1, requires_grad= True)
b = torch.randn(1, requires_grad= True)

# 前向传播，计算图，分步计算
y = torch.mul(x, w)
z = torch.add(y, b)

# 查看是否为is_leaf叶子节点
print('x,y,z,w,b是否为叶子节点:{},{},{},{},{}'.format(x.is_leaf, y.is_leaf, z.is_leaf, w.is_leaf, b.is_leaf))

# 查看子节点x, w, b的requires_grad属性
print('x,w, b 的属性为{},{},{}'.format(x.requires_grad, w.requires_grad, b.requires_grad))

# 查看非叶子节点的requires_grad梯度属性
print('y, z 的 require_grad属性：{},{}'.format(y.requires_grad, z.requires_grad)) #与w, b有依赖关系

# 查看叶子节点的grad_fn属性
print('x, w, b grad_fn is {},{},{}'.format(x.grad_fn, w.grad_fn, b.grad_fn))

# 查看非叶子节点的grad_fn属性
print('y, z grad_fn is {},{}'.format(y.grad_fn, z.grad_fn))

# 反向传播计算梯度
# z.backward() #此时不保留图graph，梯度清零
z.backward(retain_graph = True) # 梯度累加 ？

# 叶子节点：参数w b梯度, x无需求导 故为none
print('w, b grad is{},{}'.format(w.grad, b.grad))
# 非叶子节点 y z 梯度
print('y, z grad is {}, {}'.format(y.grad, z.grad))

x,y,z,w,b是否为叶子节点:True,False,False,True,True
x,w, b 的属性为False,True,True
y, z 的 require_grad属性：True,True
x, w, b grad_fn is None,None,None
y, z grad_fn is <MulBackward0 object at 0x107a4e450>,<AddBackward0 object at 0x107a4e4d0>
w, b grad istensor([2.]),tensor([1.])
y, z grad is None, None


In [3]:
# 返回，自动求导
# z.backward(retain_graph= True) 保留图，否则运行一次缓存被释放
z.backward()
# 看梯度
print('参数w,b的梯度为:{},{}'.format(w.grad, b.grad))

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [4]:
torch.__version__

'1.5.0'